<a href="https://colab.research.google.com/github/Zeilion/simulation-crowd/blob/main/simulation_crowd_notebook_v0_4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
# **Simulation de financement participatif v0.4.0**
*Prototype Pandas*
***
Date: 28/03/2022  
Author: Matthieu PELINGRE  

### Avant propos :

Le but de ce notebook est de tester une façon d'estimer la réussite du financement participatif d'un projet en fonction de différentes variables.  

L'objectif est de simuler le plus grand nombre de scénarii de vente possible atteignant un certain palier de financement. Si la totalité de l'univers des possibles est parcourue, le scénario réel, obtenu si le palier en question est atteint lors du financement participatif, sera donc forcément représenté (hors dons supplémentaires des participants).
En fonction d'une liste de coûts fixes rentré par l'utilisateur, il est alors possible d'estimer la proportion des sénarii pouvant atteindre le total de ces coûts fixes (réussite) une fois les différentes charges (part de la plateforme, frais de port, coût des contreparties) déduites du chiffre d'affaires (= montant du palier).

Si l'utilisateur estime que le taux de réussite est trop faible, il pourra alors modifier le montant du palier, le prix des contreparties, ou renégocier certains coûts fixes, pour espérer rentrer dans ses frais, selon un seuil de tolérance dont lui seul peut être tenu pour responsable.
Pour l'aider dans cette décision, divers outils peuvent être utilisés :
- la description statistique des essais échoués et réussis ;
- le simulateur permet également d'estimer le montant optimal d'un palier, pour un profit et un taux de réussite fixés par l'utilisateur;
- l'affichage de graphs de corrélations (ex. impact d'une contrepartie donnée sur les profits réalisés).

*NB: L'utilisation de Numpy et Pandas pour la création, le stockage et le traitement des données permet de réduire le temps d'exécution à 20s contre 1m32s avec le précédent prototype utilisant des dictionnaires  pour 1 million d'essais (ou 2s contre 17s pour 100 000 essais).*

### Instructions :

- appuyez sur *Ctrl+F9* pour exécuter tout le code de la page
- allez dans la section **Affichage** (en bas de page) pour interagir avec l'interface graphique
- ajoutez/supprimez des contreparties à la simulation à l'aide des boutons "+" et "-"
- rentrez vos paramètres de simulation 
- cliquez sur le bouton "Calculer" pour lancer la simulation (plus le nombre d'essais est important, plus le temps d'exécution sera long)
- rentrez la liste des coûts dont le total déterminera la réussite du financement et cliquez sur le bouton "Actualiser" pour afficher le graphique
- vous pouvez modifier la liste des coûts à tout moment et cliquer sur "Actualiser" pour afficher la nouvelle limite de réussite (total des coûts fixes) en générant un nouveau graphique
- si vous souhaitez ajouter de nouvelles contreparties, il vous faudra alors recalculer la simulation avant d'afficher un nouveau graph (nous vous conseillons, lors de ces phases exploratoires, de rester à des nombres d'essais de l'ordre de 10 000)
- une fois le graph généré, il pourra être téléchargé à l'aide du bouton "Save png"
- vous pouvez également télécharger, au format .xlsx, les paramètres que vous avez saisi à l'aide du bouton "Télécharger", et les Téléverser à l'aide du bouton correspondant pour reprendre plus facilement le travail

### Aides :

- vous pouvez cacher la section **Code** en cliquant sur la flèche à gauche du titre de section
- laissez votre souris sur les descriptions des différents champs pour plus d'information sur les paramètres associés
- le champ "Limite" d'une contrepartie correspond au nombre maximal de ventes possible pour cette contrepartie en cas d'édition limitée, si la valeur du champ est 0, aucune limite n'est fixée 
- sur le graphique, vous verrez affiché un pourcentage de doublons, plus celui-ci est élevé, plus votre simulation aura parcouru l'univers des sénarii de ventes possibles
- il est déconseillé de modifier manuellement les .xlsx générés sous peine de perte de compatibilité (à moins de savoir ce que vous faites et de garder une structure similaire au fichier de départ, sans changer les labels des tableaux et des sheets)

### Changelog :

#### *v0.4.0*

- correction de l'affichage de l'UI avec désactivation du scrolling automatique de l'output si sa taille dépasse les 1000px
- ajout des description statistique des essais échoués et réussis
- affichage de graphs de corrélations (ex. impact d'une contrepartie donnée sur les profits réalisés)
- modification de l'ordre des boîtes pour plus d'ergonomie

#### *v0.3.0*

- modification d'affichage du graph : ajout des ventes mini, moyenne et maxi
- ajout d'un bouton pour télécharger le graph
- estimation d'un palier atteignant un certain profit selon un taux de réussite fixé par l'utilisateur

#### *v0.2.6*

- possibilité de télécharger/téléverser les paramètres de simulation au format xlsx

#### *v0.2.5*





- La liste des coûts devient modifiable et peut être étendue comme les contreparties.
- Correction de l'affichage du titre et des labels des axes du graphique principal (attribués sur ax et non plus sur fig).
- affichage des paramètres de simulation à gauche du graph et la liste des coûts à droite.

***
# **Code**

***
## Modules

In [ ]:
#@title
# réinitialise l'espace de travail (supprime les fichiers)
!rm *
# modules
import time
import numpy as np
import pandas as pd
import seaborn as sns
import ipywidgets as wg
import matplotlib.pyplot as plt
from IPython.display import display
from google.colab import files

rm: cannot remove 'sample_data': Is a directory


désactivation du scrolling automatique pour les outputs trop grand

In [ ]:
#@title
from IPython.display import Javascript
# utiliser display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

***
## Classes

Classe `Counterpart`, permet de stocker et calculer les différentes informations pour une contrepartie.

In [ ]:
#@title
class Counterpart:
    """project counterparts"""
    
    def __init__(self, price, cost=0, limit=0, weight=1):
        """
        Attributes initialisation :
        - price
        - cost
        - profit (= price - cost)
        - limit: number of CP available (0 if no limit)
        - weight: TODO voir comment implémenter
        """
        # TODO : tests de compatibilité/retour d'exceptions à faire
        self.price = price
        self.cost = cost
        self.profit = price - cost
        self.limit = limit  # number of CP available, 0 if no limit
        self.weight = weight  # TODO : voir comment implémenter ce poids dans le code
    
    def __repr__(self):
        """Representation of counterparts as: CP(price, cost, limit, weight)"""
        return f'CP({self.price}, {self.cost}, {self.limit}, {self.weight})'
    
    def __iter__(self):
        """Makes object iterable and returns, in order : price, cost, limit, weight"""
        res_list = [self.price, self.cost, self.limit, self.weight]
        for i in res_list:
            yield i
    
    def __bool__(self):
        """Return True if profits are over 0€"""
        return self.profit > 0
    
    def __eq__(self, other):
        """Define self == other"""
        res = True
        for i, j  in zip(self, other):
            if i != j:
                res = False
                break
        return res
    
    def __ne__(self, other):
        """Define self != other"""
        res = False
        for i, j  in zip(self, other):
            if i != j:
                res = True
                break
        return res
    
    def __lt__(self, other):
        """Define self < other"""
        return self.profit < other.profit
    
    def __le__(self, other):
        """Define self <= other"""
        return self.profit <= other.profit
    
    def __gt__(self, other):
        """Define self > other"""
        return self.profit > other.profit
    
    def __ge__(self, other):
        """Define self >= other"""
        return self.profit >= other.profit
    
    def __hash__(self):
        """Define hash value (WIP)"""
        # TODO : revoir si redéfinition de weight
        return (self.profit * 100 + self.limit) * (self.price * 100) * int(self.weight * 100)
    
    def __call__(self, number):
        """Makes CP callable, return total profits calculated from a certain number x : CP(x) = x * profits"""
        return number * self.profit
    
    # TODO : Méthodes

Classe `Containers_gen` pour permettre la génération des différents tableaux utilisé dans la simulation et les opérations associées.

In [ ]:
#@title
class Containers_gen:
    """containers and operations for trials"""

    def __init__(self, trials_nb, cp_list, milestone, 
                 plateform_part=0, shipping_fees=0, just_milestone=False):
        """initialise the different containers needed for all operations :
        - self.dict_cp: dict of 'CP_X' and the associate 'Counterpart'
        - self.trials_df: DataFrame, will store the results of the trials
        - self.revenues: Serie, will store the total sales performance
        - self.shipping_fees: float, shipping fee for one sell
        - self.plateform_part: float, plateforme participation percentage
        - self.milestone: int, milestone to reach
        """
        cp_nb = len(cp_list)
        # dataframe wich will store trials
        self._index_cp = [f'CP_{x}' for x in range(1, cp_nb + 1)]
        self.trials_df = pd.DataFrame(np.zeros((trials_nb, cp_nb)),
                                      columns=self._index_cp)
        # array wich will be randomized
        self._random_ar = np.zeros((trials_nb, cp_nb))  # tableau vide doit être généré une fois
        self._random_ar[:, 0] = 1 
        # serie wich will store the total sales performance
        self.revenues = pd.Series(np.empty((trials_nb)), index=range(trials_nb))
        # mask wich will be use for revenues < MILESTONE
        self._ts_mask = pd.Series(np.empty((trials_nb)), index=range(trials_nb))
        # random generator
        self._rng = np.random.default_rng()
        # conterparts dictionnary
        self.dict_cp = dict(zip(self._index_cp, cp_list))
        # shipping fees
        self.shipping_fees = shipping_fees
        # plateform participation percentage
        self.plateform_part = plateform_part
        # store milestone
        self.milestone = milestone
    
    def reset_containers(self):
        """Reset trials_df and revenues"""
        self.trials_df = self.trials_df * 0
        self.revenues = self.revenues * 0

    def sell_rnd_one(self, just_milestone=False):
        """randomly increments a CP, for each trial, by one sale if the limit of
        this CP is not reached or if the milestone is not reached on the 
        considered trial"""
        # randomize ones positions in random_array
        self._rng.permuted(self._random_ar, axis=1, out=self._random_ar)

        # calculate price list of CPs
        p_list = sorted(cp.price for cp in self.dict_cp.values())

        # create the mask
        self._ts_mask = self.revenues < self.milestone

        # if we admit that people will just buy the rest for the last sell
        if just_milestone:
            self.revenues = (self.trials_df + self._random_ar) @ p_list
            self._ts_mask = self.revenues <= self.milestone

        # add the sells where it doesn't reach the MILESTONE
        self.trials_df[self._ts_mask] += self._random_ar[self._ts_mask]

        # sell limits
        for key, cp in self.dict_cp.items():
            if cp.limit:  # si une limite est fixée
                tmp_view = self.trials_df.loc[:, key]
                tmp_view[tmp_view > cp.limit] = cp.limit  

        # calculate the revenues
        self.revenues = self.trials_df @ p_list
    

    def sell_to_milestone(self, just_milestone=False):
        """Applies the sell_rnd_one method as long as all the trials have not 
        reached the milestone.
        If the trials_df as already been filled, everything is reset for another
        trial.
        ---------
        If the boolean just_milestone is set to True, we'll admit that people 
        will just buy what's needed to reach the milestone for the last sell
        (the calculation time will be increase)."""
        # reset if needed
        if np.any(self.revenues >= self.milestone):
            self.reset_containers()
        # proscess
        while np.any(self.revenues < self.milestone):
            self.sell_rnd_one(just_milestone)


    # The next operations are implemented as method because they are not needed
    # until the end of a full cycle of the sell_to_milestone() method
    # Theses methods are only used in df_result() method
    def _sells_nb(self):
        """calculates the total number of CPs sold per trial"""
        return pd.DataFrame({'sells' : np.sum(self.trials_df, axis=1)})

    def _costs(self):
        """calculates the total costs for each trial"""
        c_list = [(self.dict_cp[cp]).cost for cp in self._index_cp]
        return pd.DataFrame({'costs' : self.trials_df @ c_list})

    def _ship_fees(self):
        """calculates the shipping fees for each trial"""
        return pd.DataFrame({'shipping': (self.shipping_fees 
                                          * self._sells_nb()['sells'])})
    
    def _plateform(self):
        """calculate the plateform fee for each trial"""
        return pd.DataFrame({'plateform': self.revenues * self.plateform_part})

    def _profits(self):
        """calculates the profit made for each trial
        (deduce the shipping fees)"""
        profit_list = [(self.dict_cp[cp]).profit for cp in self._index_cp]
        return pd.DataFrame({'profits' : (self.trials_df @ profit_list)
                                          - self._ship_fees()['shipping']
                                          - self._plateform()['plateform']})


    def df_result(self):
        """Build the full DataFrame of results in full_df attribute, clean 
        duplicates and create the duplicate_nb and duplicate_percent attributes
        """
        # full dataframe building
        self.full_df = (pd.concat([self.trials_df,
                                   self._sells_nb(),
                                   pd.DataFrame({'revenues' : self.revenues}),
                                   self._costs(),
                                   self._ship_fees(),
                                   self._plateform(),
                                   self._profits(),
                                   ], axis=1)
                          .drop_duplicates()
                          .sort_values('profits')
                          .reset_index(drop=True))
        
        # duplicate number and percentage
        self.duplicate_nb = len(self.trials_df) - len(self.full_df)
        self.duplicate_percent = self.duplicate_nb / len(self.trials_df)

***
## Tableau de bord

### Paramètres widgets :

In [ ]:
#@title
# contôle de la taille des widgets
l_100 = wg.Layout(width='100%')
l_75 = wg.Layout(width='73%')
l_66 = wg.Layout(width='64%')
l_50 = wg.Layout(width='48%')
l_33 = wg.Layout(width='31%')
l_25 = wg.Layout(width='23%')
l_20 = wg.Layout(width='19%')
l_12 = wg.Layout(width='12%')
l_style = {'description_width': 'initial'}  # permet de compter la descritption dans la taille
l_spacebtw = wg.Layout(justify_content='space-between')  # espace les éléments entre eux
l_center = wg.Layout(justify_content='center')

### Boîte de paramètres pour le calcul de la DataFrame (`box_df`) :

#### Boîte de paramètres du financement participatif (`top_param_box`) :

In [ ]:
#@title
# Boîte de paramètres du financement participatif
# MILESTONE, TRIALS_NB, PLATEFORME_PART, SHIPPING_FEES

top_line_label = wg.HTML(value=f"<b>Paramètres du financement :</b>",
                         layout=l_20,
                         style=l_style,
                         )

w_milestone = wg.BoundedIntText(value=2_000, min=0, 
                                max=1_000_000_000,
                                step=100,
                                description="Palier",
                                layout=l_20,
                                style=l_style,
                                disabled=False,
                                description_tooltip='Palier de financement devant être atteint lors de la simulation.',
                                )

w_trials_nb = wg.BoundedIntText(value=10_000, min=10_000, 
                                max=1_000_000_000,
                                step=10_000,
                                description="Essais",
                                layout=l_20,
                                style=l_style,
                                disabled=False,
                                description_tooltip='Nombre d\'essais à réaliser lors de la simulation.',
                                )

w_plateform_part = wg.BoundedFloatText(value=5, min=0, max=100, step=0.5,
                                       description='% Plateforme',
                                       layout=l_20,
                                       style=l_style,
                                       disabled=False,
                                       description_tooltip='Part qui sera prélevée par la plateforme en pourcentage du chiffre d\'affaires.',
                                       )

w_shipping_fees = wg.BoundedFloatText(value=0, min=0, max=1_000_000_000, 
                                      step=0.5,
                                      description='Frais de port',
                                      layout=l_20,
                                      style=l_style,
                                      disabled=False,
                                      description_tooltip='Estimation des frais de port, en €, déduit lors d\'une vente.',
                                      )

top_param_box = wg.HBox([top_line_label, w_milestone, w_trials_nb, 
                         w_plateform_part, w_shipping_fees], 
                        layout=wg.Layout(justify_content='space-between',
                                         border='1px solid'))

#### Boîte de paramètres des contreparties (`cp_box`) :


Ajouter les poids stats une fois implémentés

In [ ]:
#@title
# Counterparts
# ============

# compteur CP (TODO voir si mieux ?)
cp_count = 1

def fw_bfloattext(val=0, desc='Desc', desc_tt='Desc'):
    """Crée un widget BoundedFloatText"""
    return wg.BoundedFloatText(value=val, min=0, max=1_000_000_000, step=0.5,
                               description=desc,
                               layout=wg.Layout(width='23%'),
                               style=l_style,
                               disabled=False,
                               description_tooltip=desc_tt,
                               )

def fw_binttext(val=0, desc='Desc', desc_tt='Desc'):
    """Crée un widget BoundedIntText"""
    return wg.BoundedIntText(value=val, min=0, max=1_000_000_000, 
                             description=desc,
                             layout=l_25,
                             style=l_style,
                             disabled=False,
                             description_tooltip=desc_tt,
                             )

def fw_cp_line(i):
    """Crée une line de widget pour une CP_i"""
    line_label = wg.HTML(value=f"<b>Contrepartie {i}:</b>",
                         layout=l_25,
                         style=l_style,
                         )
    return wg.HBox([line_label,
                    fw_bfloattext(10, 'Prix', 'Prix de vente de la contrepartie'), 
                    fw_bfloattext(5, 'Coût', 'Coût matériel de la contrepartie'), 
                    fw_binttext(0, 'Limite', 'Nombre de contreparties disponible, 0 si pas de limite')],
                   layout=l_spacebtw)

# première ligne CP
first_cp_line = fw_cp_line(cp_count)
# ajout première ligne dans la première colonne (boîte des CP)
cp_box = wg.VBox([first_cp_line])

# boutons add/remove
add_btn = wg.Button(description = '+', 
                    button_style='',
                    tooltip="Ajout d'une contrepartie.",
                    layout=l_12,
                    )

rm_btn = wg.Button(description = '-', 
                   button_style='',
                   tooltip="Supprime la dernière contrepartie.",
                    layout=l_12,
                   )

def on_add_clicked(b):
    global cp_count, actualise_btn
    reset_calc_btn()  # reset bouton de calcul
    reset_load_bar()  # reset load_bar
    if actualise_btn:
        actualise_btn.disabled = True  # désactive le bouton d'acutalisation
    cp_count += 1
    cp_box.children=tuple(list(cp_box.children) + [fw_cp_line(cp_count)])
    b.button_style='success'
    time.sleep(1)
    b.button_style=''

def on_rm_clicked(b):
    global cp_count, actualise_btn
    reset_calc_btn()  # reset bouton de calcul
    reset_load_bar()  # reset load_bar
    if actualise_btn:
        actualise_btn.disabled = True  # désactive le bouton d'acutalisation
    if cp_count > 1:
        cp_count -= 1        
        cp_box.children=tuple(list(cp_box.children)[:-1])
        b.button_style='success'
        time.sleep(1)
        b.button_style=''
    else:
        b.button_style='danger'
        time.sleep(1)
        b.button_style=''


add_btn.on_click(on_add_clicked)
rm_btn.on_click(on_rm_clicked)

Barre de chargement calcul df

In [ ]:
#@title
load_bar = wg.IntProgress(
    value=0,  # à modifier pendant le calcul
    min=0,
    max=w_trials_nb.value,  # à modifier avant le calcul
    description='Chargement',  # à changer par 'Complete' à la fin du calcul
    style={'bar_color': 'maroon'},
    orientation='horizontal',
    layout=wg.Layout(width='50%',
                     visibility='hidden')  # n'afficher que lors du calcul
)

# fonction de reset de la barre
def reset_load_bar():
    """Lors de changement au niveau des CP"""
    global load_bar
    load_bar.layout.visibility='hidden'
    load_bar.value=0

Récupération des données et calcul DataFrame

In [ ]:
#@title
# bouton de calcul
calc_btn = wg.Button(description = 'Calculer', 
                     button_style='',
                     tooltip="Calcul des données.",
                     layout=l_25,
                     )

# fonction de reset du bouton
def reset_calc_btn():
    """Lors de changement au niveau des CP"""
    global calc_btn
    calc_btn.description = 'Calculer'
    calc_btn.button_style=''
    calc_btn.tooltip="Calcul des données."

# déclaration container
contain = None

# fonction associée
def on_calc_clicked(b):
    global contain, actualise_btn
    
    # reset le style du bouton
    reset_calc_btn()
    # désactive le bouton pendant le calcul
    b.disabled = True

    # reset barre de chargement
    reset_load_bar()
    load_bar.max = w_trials_nb.value

    # récupération valeurs
    # ====================

    # contreparties :
    liste_cp = []  # liste des CP de générés avec la classe couterparts
    for line in list(cp_box.children):
        liste_cp.append(Counterpart(line.children[1].value, 
                                    line.children[2].value, 
                                    line.children[3].value,))

    # crée les contenants
    contain = Containers_gen(w_trials_nb.value, 
                             liste_cp,
                             w_milestone.value,
                             plateform_part=(w_plateform_part.value / 100), 
                             shipping_fees=w_shipping_fees.value)
    # reset if needed
    contain.reset_containers()
    # affiche la barre de chargement
    load_bar.layout.visibility = 'visible'
    # on fait l'équivalent de contain.sell_to_milestone() pour la barre de chargement
    while np.any(contain.revenues < contain.milestone):
        contain.sell_rnd_one()  # une vente
        # chargement :
        load_bar.value = np.count_nonzero(contain.revenues >= w_milestone.value)
    contain.df_result()  # calcul des résultats
    # bouton fin
    b.disabled = False  # réactive le bouton après le calcul
    b.description='Terminé'
    b.button_style='success'
    b.tooltip='Cliquer pour recalculer.'
    if actualise_btn:
        actualise_btn.disabled=False
        actualise_btn.tooltip="Actualise le graphique."

    # met à jour le profit voulu dans la section d'estimation
    actualise_desired_profit()

calc_btn.on_click(on_calc_clicked)

#### Assemblage de (`box_df`) :

In [ ]:
#@title
# assemblage de box_df :
box_df = wg.VBox([top_param_box, 
                  cp_box, 
                  wg.HBox([add_btn, rm_btn, load_bar, calc_btn],
                          layout=l_spacebtw,
                          ),
                  ],
                 layout=wg.Layout(border='1px solid'),
                 )

### Boîte des graphs (`box_graph`) :

#### Fonction de tracé `graph_plot`



In [ ]:
#@title
# on crée la sortie du graph
out_graph = wg.Output(layout=wg.Layout(width='100%',
                                       justify_content='center',
                                       ))

# capture de la sortie
@out_graph.capture()
def graph_plot(fixed_costs=[500], percent_costs=[25]):
    """fonction traçage du graph
    avec la limite : fixed_costs + percent_costs * contain.milestone"""
    global contain
    # obligé d'utiliser contain en global pour les widgets

    if contain:
        # ferme toutes les instances de pyplot avant d'en ouvrir une nouvelle
        plt.close('all')  

        # vue sur les profits par essais
        view_profits = contain.full_df.loc[:, 'profits']

        # création de la figure et du subplot
        fig, (pline1, pline2) = plt.subplots(2, 3, 
                                             figsize=(19.6, 9.6),
                                             constrained_layout=True)
        
        # première ligne de plots :
        (l_ax, ax, r_ax) = pline1
        # deuxième ligne :
        (lb_ax, mb_ax, rb_ax) = pline2

        # ======== #
        # Axe = ax #
        # ======== #

        # affichage limite
        limit = sum(fixed_costs) + (sum(percent_costs)/100) * contain.milestone
        limit_serie = pd.Series([limit for x in range(len(view_profits))])
        limit_serie.plot(ax=ax, color='r')
        # annotation de la limite
        ax.text(view_profits.index.max() * 0.99, limit,
                f"coûts fixes", color='red',
                ha='right', va='center',
                bbox=dict(boxstyle="round,pad=0.1", fc='w', ec='r'))
                
        # vue sur essais ayant atteint l'objectif
        view_success = contain.full_df[contain.full_df.loc[:, "profits"] >= limit]

        # vue sur essais n'ayant pas atteint l'objectif
        view_failed = contain.full_df[contain.full_df.loc[:, "profits"] < limit]
        
        # affiche un rectangle gris entre les essais 0 et celui profit > limit
        if len(view_failed.index):  # si des sénarii ont échoués
            if len(view_success.index):
                y_rect = view_profits
            else:
                y_rect = [view_profits[0], limit]
            ax.fill_betweenx(y=y_rect,  # les y du rectangle
                            x1=view_failed.index[-1],  # le x de fin (x2=0 par defaut)
                            color='grey', alpha=0.2)
            # annotation du rectangle
            ax.text(0.05, 0.95, 
                    f"échecs", color='grey', transform=ax.transAxes,
                    ha='left', va='top')


        # trace les données dans le subplot
        ax.scatter(view_profits.index, view_profits, s=2, color='g', lw=2.5)
        # titre et y label :
        ax.set_title(f"Gains sur {len(view_profits)} scénarii au palier de {contain.milestone}€\n"
                     f"({contain.duplicate_percent:.2%} de doublons, "
                     f"{len(view_success)/len(view_profits):.2%} de réussites)")
        ax.set_ylabel(f'Bénéfices bruts (€)')
        ax.set_xlabel(f'Essais')

        # affiche la valeur du gains minimum
        ax.text(view_profits.index.max() * 0.01, view_profits.min(), 
                f"{view_profits.min():.2f}€")
        ax.text(view_profits.index.max() * 0.99, view_profits.max(), 
                f"{view_profits.max():.2f}€", 
                ha='right', va='top')


        # ========== #
        # Axe = l_ax #
        # ========== #

        l_ax.set_axis_off()
        l_ax.set_title(f"Paramètres de simulation")
        l_ax.text(0.05, 0.95, (txt_crowd() + "\n\n" + txt_cp()), 
                  transform=l_ax.transAxes,
                  ha='left', va='top',)

        # ========== #
        # Axe = r_ax #
        # ========== #

        r_ax.set_axis_off()
        r_ax.set_title(f"Autres informations")
        r_ax.text(0.05, 0.95, (txt_fcosts() + "\n\n" + txt_sells(contain)), 
                  transform=r_ax.transAxes,
                  ha='left', va='top',)

        
        # =========== #
        # Axe = lb_ax # description stat essais échoués
        # =========== #

        # y min et max pour lb_ax et rb_ax
        y_max = contain.full_df.iloc[:, 0:len(contain.dict_cp)].max().max() + 1
        y_min = contain.full_df.iloc[:, 0:len(contain.dict_cp)].min().min() - 1

        lb_ax.set_title(f"Description des échecs")
        view_failed.iloc[:, 0:len(contain.dict_cp)].boxplot(ax=lb_ax)
        lb_ax.set_xlabel('Contreparties')
        lb_ax.set_ylabel('Ventes')
        lb_ax.set_ylim(y_min, y_max)

        # =========== #
        # Axe = mb_ax # description des boxplot ?
        # =========== #

        mb_ax.set_axis_off()
        mb_ax.set_title(f"Boîtes à moustaches (box plot) - explications", y=0.93)
        def_boxplot = ("Un box plot est une méthode permettant de représenter graphiquement\n"
                       "des groupes de données numériques par le biais de leurs quartiles.\n"
                       "\n"
                       "En statistique descriptive, un quartile est chacune des trois valeurs\n"
                       "qui divisent les données triées en quatre parts égales, de sorte que\n"
                       "chaque partie représente 1/4 de l'échantillon de population.\n"
                       "\n"
                       "La boîte s'étend des valeurs des quartiles Q1 à Q3 des données*, avec\n"
                       "une ligne à la médiane (Q2).\n"
                       "Les moustaches s'étendent à partir des bords de la boîte pour montrer\n"
                       "l'étendue des données. Par défaut, elles ne s'étendent pas au-delà de\n"
                       " 1,5 x IQR (IQR = Q3 - Q1) à partir des bords de la boîte, se terminant\n"
                       "au point de données le plus éloigné dans cet intervalle.\n"
                       "\n"
                       "Les valeurs aberrantes sont représentées par des points séparés.\n"
                       "\n"
                       "\n"
                       "*Q1 est la donnée de la série qui sépare les 25% inférieurs des données\n"
                       "*Q3 est la donnée de la série qui sépare les 75% inférieurs des données\n"
                       "    -> 50% des données seront donc comprises entre Q1 et Q3")
        mb_ax.text(0.05, 0.90, def_boxplot, 
                  transform=mb_ax.transAxes,
                  ha='left', va='top',)

        # =========== #
        # Axe = rb_ax # description stat essais réussis
        # =========== #

        rb_ax.set_title(f"Description des réussites")
        view_success.iloc[:, 0:len(contain.dict_cp)].boxplot(ax=rb_ax)
        rb_ax.set_xlabel('Contreparties')
        rb_ax.set_ylabel('Ventes')
        rb_ax.set_ylim(y_min, y_max)

        # sauvegarde image
        fig.savefig('graph.png')
        # affiche l'image finale
        plt.show();

#### Sous-boîte de la liste des coûts  `vb_fcost_list`

In [ ]:
#@title
# Liste des coûts (vb_fcost_list)
# ===============

# compteur de coûts (TODO voir si mieux ?)
fcost_count = 1

def fw_fcost_name(val=1):
    """Crée un widget Text pour la référence des coûts"""
    return wg.Text(
        value=f'Coût {val}',
        placeholder='Type something',
        description='Référence',
        layout=l_33,
        style=l_style,
        description_tooltip=('Référence du coût, la somme de la liste des coûts '
                             'détermine la réussite du financement.'),
        )

def fw_fcost(val=0):
    """Crée un widget BoundedIntText pour les coûts fixes"""
    return wg.BoundedIntText(
        value=val, min=0, 
        max=1_000_000_000,
        step=50,
        description="Coût fixe",
        layout=l_33,
        style=l_style,
        disabled=False,
        description_tooltip=('Coût fixe (illustration, ISBN ...) ne dépendant pas '
                            'de la valeur du palier'
                            '(est ajouté à "+ % du palier").'),
        )

def fw_pcost(val=0):
    """Crée un widget BoundedIFloatText pour les coûts en % du palier atteint"""
    return wg.BoundedFloatText(
        value=val, min=0, max=100, step=0.5,
        description='+ % du palier',
        layout=l_33,
        style=l_style,
        disabled=False,
        description_tooltip=('Coût en pourcentage du palier atteint '
                            '(est ajouté à "Coûts fixes").'),
        )

def fw_fcost_line(i, valf=0, valp=0):
    """Crée une line de widget pour un coût"""
    return wg.HBox([fw_fcost_name(i),
                    fw_fcost(val=valf), 
                    fw_pcost(val=valp),
                    ],
                   layout=l_spacebtw)

# première ligne des coûts
first_fcost_line = fw_fcost_line(fcost_count, valf=500, valp=25)
# ajout de la première ligne dans le conteneur des coûts auquel il faudra ajouter des lignes
vb_fcost_list = wg.VBox([first_fcost_line,  # ligne 1
                         ],  # ligne 2 ...
                        )

#### Ligne des boutons `hb_fcost_buttons`

Boutons add/rm et fonctions associées :

In [ ]:
#@title
# boutons add/remove
add_fcost_btn = wg.Button(
    description = '+', 
    button_style='',
    tooltip="Ajout d'un coût supplémentaire.",
    layout=l_50,
    )

rm_fcost_btn = wg.Button(
    description = '-',
    button_style='',
    tooltip="Supprime la dernière ligne de coûts.",
    layout=l_50,
    )


def on_add_fcost_clicked(b):
    global fcost_count
    fcost_count += 1
    vb_fcost_list.children=tuple(list(vb_fcost_list.children) + [fw_fcost_line(fcost_count)])
    b.button_style='success'
    time.sleep(1)
    b.button_style=''

def on_rm_fcost_clicked(b):
    global fcost_count
    if fcost_count > 1:
        fcost_count -= 1        
        vb_fcost_list.children=tuple(list(vb_fcost_list.children)[:-1])
        b.button_style='success'
        time.sleep(1)
        b.button_style=''
    else:
        b.button_style='danger'
        time.sleep(1)
        b.button_style=''


add_fcost_btn.on_click(on_add_fcost_clicked)
rm_fcost_btn.on_click(on_rm_fcost_clicked)

Bouton "Actualiser"

In [ ]:
#@title
# interaction entre les widgets et la fonction d'affichage
actualise_btn =  wg.Button(
    description = 'Actualiser', 
    button_style='',
    tooltip="Calculez les données avant de générer le graph.",
    layout=wg.Layout(width='28.6%'),
    disabled=True, # on ajoute actualise_btn.disable = False lors génération contain
)
# le bouton doit aussi être désactivé lors de la modification des CP (boutons)

def on_actualise_clicked(b):
    global contain, out_graph
    out_graph.clear_output()  # on nettoie l'output

    # récupération valeurs des coûts
    fcost_val = []  
    pcost_val = []
    for line in list(vb_fcost_list.children):
        fcost_val.append(line.children[1].value)
        pcost_val.append(line.children[2].value)

    # tracé
    if contain:
        graph_plot(fcost_val, pcost_val)
        b.button_style='success'
        time.sleep(1)
        b.button_style=''
        # on active la possibilité de sauvegarder l'image
        save_image_btn.disabled=False
    else:
        b.button_style='danger'
        with out_graph:
            print(' Erreur : les données n\'ont pas été générées.')
        time.sleep(1)
        b.button_style=''

    # met à jour le profit voulu dans la section d'estimation
    actualise_desired_profit()

actualise_btn.on_click(on_actualise_clicked)

Bouton de téléchargement d'image

In [ ]:
#@title
# interaction entre les widgets et la fonction d'affichage
save_image_btn =  wg.Button(
    description = 'Save png', 
    button_style='',
    tooltip="Générez d'abord le graphique.",
    layout=wg.Layout(width='15%'),
    disabled=True, # activé si actualise_btn est cliqué, désactivé si désactivé également
)
# cf on_actualise_clicked()

# désactivation si actualise_btn est désactivé
def on_actualise_change(change):
    if change['new']:
        save_image_btn.disabled = change['new']
actualise_btn.observe(on_actualise_change, names='disabled')

# changement text à l'activation/désactivation du bouton
def on_save_image_change(change):
    if change['new']:
        save_image_btn.tooltip="Générez d'abord le graphique."
    else:
        save_image_btn.tooltip="Sauvegarde le graphique au format png."
save_image_btn.observe(on_save_image_change, names='disabled')

def on_saveimg_clicked(b):
    global contain, out_graph

    # tracé
    if contain:
        files.download('graph.png')
        b.button_style='success'
        time.sleep(1)
        b.button_style=''
    else:
        b.button_style='danger'
        with out_graph:
            print(' Erreur : les données n\'ont pas été générées.')
        time.sleep(1)
        b.button_style=''

save_image_btn.on_click(on_saveimg_clicked)

Ligne des boutons

In [ ]:
#@title
# séparateur
w_sep = wg.HTML(
    value='',
    layout=wg.Layout(width='22.6%'),
    style=l_style,
    )

# ligne des boutons
hb_fcost_buttons = wg.HBox([wg.HBox([add_fcost_btn, rm_fcost_btn], 
                                    layout=wg.Layout(width='31.5%', 
                                                     justify_content='space-between', 
                                                     )), w_sep, save_image_btn, actualise_btn], 
                           layout=l_spacebtw)

#### Construction de `box_graph`

In [ ]:
#@title
# label de la boîte des coûts
fcost_label = wg.HTML(
    value=f"<b>Liste des coûts :</b>",
    layout=l_20,
    style=l_style,
    )

# boîte des coûts fixes
box_fcost = wg.HBox([fcost_label, wg.VBox([vb_fcost_list,  
                                           hb_fcost_buttons,
                                           ], layout=wg.Layout(width='79.9%')),
                     ], layout=wg.Layout(justify_content='space-between', border='1px solid'))

# box_graph
box_graph = wg.VBox([box_fcost,
                     wg.HBox([out_graph], layout=l_center),
                     ],
                   layout=wg.Layout(border='1px solid'))

***
## Gestion des paramètres

### Extraction des paramètres

In [ ]:
#@title
# extraction des paramètres fixes de la simulation
def ext_crowd(crowd_widgts=top_param_box):
    """
    Extrait les paramètres fixes de la simulation (palier, nombre d'essais, part 
    de la plateforme et frais de port) des valeurs des widgets correspondants
    dans la boîte top_param_box
    """
    crowd_data = {}
    for widgt in list(crowd_widgts.children)[1:]:  # car en 0 : label
        crowd_data[widgt.description] = widgt.value
    return pd.DataFrame(crowd_data, index=['Valeurs'])

# extraction des paramètres des contreparties
def ext_cps(cp_widgts=cp_box):
    """
    Extrait les paramètres des contreparties de la simulation (prix, coût, limite) 
    des valeurs des widgets correspondants dans la boîte cp_box
    """
    cp_data = {}
    for line in list(cp_widgts.children):
        for i in range(1,  len(list(line.children))):  # len permet de prendre en compte poids si implémenté
            if line.children[i].description in cp_data:
                cp_data[line.children[i].description].append(line.children[i].value)
            else:
                cp_data[line.children[i].description] = [line.children[i].value]
    return pd.DataFrame(cp_data, 
                        index=[f'CP_{i}' 
                               for i in range(1, len(list(cp_widgts.children))+1)])
    
# extraction des paramètres des coûts
def ext_fcosts(costs_widgts=vb_fcost_list):
    """
    Extrait les paramètres des coûts de la simulation (référence, coût fixe, % palier) 
    des valeurs des widgets correspondants dans la boîte vb_fcost_list
    """
    costs_data = {}
    for line in list(costs_widgts.children):
        for i in range(len(list(list(vb_fcost_list.children)[0].children))):  # = longueur ligne
            if line.children[i].description in costs_data:
                costs_data[line.children[i].description].append(line.children[i].value)
            else:
                costs_data[line.children[i].description] = [line.children[i].value]
    return pd.DataFrame(costs_data, 
                        index=[f'Cost_{i}' 
                               for i in range(1, len(list(costs_widgts.children))+1)])
    # voir si index nécessaire ici et ne pose pas de problèmes

### Texte d'affichage des paramètres

In [ ]:
#@title
def txt_crowd(crowd_widgts=top_param_box):
    """Transforme les paramètres du financement, extrait à l'aide de 
    ext_crowd() en str pour l'affichage à gauche du graph"""
    unit_list = [" €", "", " %", " €"]
    data = ext_crowd(crowd_widgts)
    res = "Financement :\n"
    for i, column in enumerate(data.columns):
           res += f"        - {column} : {data.iloc[0, i]:,}{unit_list[i]}\n"
    # les chiffres sont représenté au format anglo-saxon (1,000.02)
    # traduction en fr :
    res = res.replace(',', ' ')
    res = res.replace('.', ',')
    return res

def txt_cp(cp_widgts=cp_box):
    """Transforme les paramètres des contreparties, extrait à l'aide de 
    ext_cp() en str pour l'affichage à gauche du graph"""
    unit_list = [" €", " €", ""]
    data = ext_cps(cp_widgts)
    res = "Contreparties :\n"
    for cp_i in range(len(data)):
        res += f"        - CP {cp_i + 1} ("
        for i, column in enumerate(data.columns):
            res += f"{column} : {data.iloc[cp_i, i]:,}{unit_list[i]}; "
        res = res[:-2] + ")\n"
    # les chiffres sont représenté au format anglo-saxon (1,000.02)
    # traduction en fr :
    res = res.replace(',', ' ')
    res = res.replace('.', ',')
    return res

def txt_fcosts(costs_widgts=vb_fcost_list):
    """Transforme les paramètres des coûts, extrait à l'aide de 
    ext_fcosts() en str pour l'affichage à gauche du graph"""
    unit_list = [" :", " € +", " % du palier +"]
    # on ajoute des '+' à la fin partout, enlevé lors ajout line à res
    data = ext_fcosts(costs_widgts)
    res = "Liste des coûts :\n"
    total = 0  # total des coûts
    for cost_i in range(len(data)):
        line = "    - "
        total_line = 0  # total d'une ligne
        for i, column in enumerate(data.columns):
            if isinstance(data.iloc[cost_i, i], str):  # si c'est la référence
                line += f"{data.iloc[cost_i, i]}{unit_list[i]} "
            elif data.iloc[cost_i, i]:  # si la valeur est non nule
                line += f"{data.iloc[cost_i, i]:,}{unit_list[i]} "
                if i == 1:  # si c'est un coût fixe
                    total_line += data.iloc[cost_i, i]
            if 'palier' in line:  # si un cout en pourcentage du palier est présent
                pcost = data.iloc[cost_i, i]*w_milestone.value/100
                total_line += pcost
        # ajout de line à res
        if 'palier' in line:  # calcul du total de la ligne si pourcentage du palier
            res += line[:-2] + f"(Total : {total_line} €)\n"
        else:
            res += line[:-2] + f"\n"
        # calcul total des coûts
        total += total_line
    # ajout ligne total des coûts
    res += f"\nTotal des coûts : {total:,} €"
    # les chiffres sont représenté au format anglo-saxon (1,000.02)
    # traduction en fr :
    res = res.replace(',', ' ')
    res = res.replace('.', ',')
    return res


def txt_sells(container):
    """Text d'estimation des ventes pour affichage à droite du graph"""
    # vue sur les ventes réalisées
    sells_view = container.full_df.loc[:, "sells"]
    res = "\nEstimation des ventes :\n"
    res += f"    - minimum : {sells_view.min()}\n"
    res += f"    - moyenne : {round(sells_view.mean(), 0)}\n"
    res += f"    - maximum : {sells_view.max()}\n"
    return res

### Sauvegarde des paramètres

In [ ]:
#@title
# liste des noms des sheets du fichier excel, sert également à vérifier l'intégrité lors du téléversement
file_sheets = ["Simulation", "Contreparties", "Coûts"]

Nom du fichier xlsx à télécharger

In [ ]:
#@title
# widget nom du fichier (lié au fichier charger -> prend son nom, cf extract_upfile())
settings_filename =  wg.Text(
        value='saved_crowd_param',
        placeholder='Type something',
        description='Fichier',
        layout=l_20,
        style=l_style,
        description_tooltip=('Nom du fichier à télécharger.'),
        )

Bouton + fonction téléchargement

In [ ]:
#@title
# fonction téléchargement
def download_savefile(b):
    """Permet de télécharger tous les paramètres de simulation """
    global settings_filename
    # nom de fichier
    output_file = settings_filename.value
    if output_file[-5:] != '.xlsx':  # si pas d'extension spécifiée, on rajoute
        output_file +=  '.xlsx'
    # dictionnaire {sheet: df}
    dict_of_dfs = {file_sheets[0]: ext_crowd(),
                   file_sheets[1]: ext_cps(),
                   file_sheets[2]: ext_fcosts()}
    # écriture
    with pd.ExcelWriter(output_file) as writer:  
        for key in list(dict_of_dfs.keys()):
            dict_of_dfs[key].to_excel(writer, sheet_name=key)
    # téléchargement
    files.download(output_file)
    return True  # peut être remplacé par autre chose (comme retourner le nom du fichier)

param_download_btn = wg.Button(
    description = 'Téléchargement', 
    button_style='',
    tooltip="Téléchargement des paramètres.",
    layout=l_20,
    )

param_download_btn.on_click(download_savefile)

### Chargement des paramètres

Vérification et création des df à charger :

In [ ]:
#@title
# vérifications
def check_integrity(loaded_dict):
    """Vérifie l'intégrité des données téléchargées et retourne l'objet testé"""
    # vérifie le dtype des colonnes des différentes df
    # la seule qui doit être égale à 'O' (object) est la colonne Référence de la sheet "Coûts"
    for sheet in file_sheets:
        temp_df = loaded_dict[sheet]
        for column in temp_df.columns:
            col_dtype = np.dtype(np.dtype(temp_df.loc[:, column]))
            if column != 'Référence' and not(col_dtype == int or col_dtype == float):
                raise ValueError(f"Wrond dtype for column '{column}' in sheet '{sheet}'. "
                                f"Expected int or float, got {col_dtype}. Check values in that column.")
            elif column == 'Référence' and col_dtype != object:
                raise ValueError(f"Wrond dtype for column '{column}' in sheet '{sheet}'. "
                                f"Expected object ('O'), got {col_dtype}. Check values in that column.")
    return loaded_dict

# chargement
def load_savefile(loaded_filename):
    """Charge en mémoire les DataFrames préalablement téléchargées dans un fichier xlsx 'filename'"""
    # nom de fichier
    input_file = loaded_filename
    if input_file[-5:] != '.xlsx':  # si pas d'extension spécifiée, on rajoute
        input_file +=  '.xlsx'
    # dictionnaire {sheet: df} résultat
    loaded_dict = {}
    loaded_sheets = pd.ExcelFile(input_file).sheet_names  # noms des sheets
    if (set(loaded_sheets) & set(file_sheets)) == set(file_sheets):  # check si le fichier est correct
    # set(loaded_sheets) == set(file_sheets) si on veux un truc 100% identique
    # on pourrait aussi imaginer ne charger que les sheets désirées
    # si elles existent : (set(loaded_sheets) & set(file_sheets)) == set(file_sheets)
        for sheet in file_sheets:  # file_sheets au lieu de loaded_sheets -> que celles qu'on veut 
            loaded_dict[sheet] = pd.read_excel(input_file, sheet, index_col=0)
            loaded_dict[sheet].fillna(value=0, inplace=True)  # remplace les valeurs manquantes par 0
        return check_integrity(loaded_dict)
    else:
        raise ValueError(f'Expected {file_sheets} in sheet names, got {loaded_sheets}.')

Extraction du fichier du widget à l'aide des précédentes fonctions

In [ ]:
#@title
def extract_upfile(uploader_value):
    """Extrait le fichier d'un widget FileUpload et lui applique la fonction load_savefile().
    uploader_value est le dictionnaire auquel on accéderait via uploader.value (uploader est un wg.FileUpload)"""
    global settings_filename
    loaded_filename = list(uploader_value.keys())[0]  # nom du fichier
    settings_filename.value = loaded_filename  # remplace la valeur du widget text par le nom du fichier
    uploaded_file = uploader_value[loaded_filename]  # on récupère la valeur du fichier, un dictionnaire
    with open(loaded_filename, "wb") as fp:
        fp.write(uploaded_file['content'])  # écriture du contenu du fichier, en bytes dans ./
    return load_savefile(loaded_filename)  # charge en mémoire le dict {sheet: df}

Modification des paramètres du tableau de bord en fonction des df chargées

In [ ]:
#@title
def load_pcrowd(crowd_settings):
    """Charge les nouveaux paramètres de simulation de crowd_settings dans top_param_box.
    crowd_settings est la DataFrame référencée par 'Simulation' dans loaded_settings."""
    global top_param_box
    for widgt in list(top_param_box.children)[1:]:  # car en 0 : label
        if widgt.description in crowd_settings.columns:
            widgt.value = crowd_settings.iloc[0].loc[widgt.description]
        else:
            raise ValueError(f"Array labels in 'Simulation' sheet ({list(crowd_settings.columns)})" 
                             f"have no '{widgt.description}' label")
    # return quelque chose ?

def load_pcp(cp_settings):
    """Charge les nouveaux paramètres des contreparties de cp_settings dans cp_box.
    cp_settings est la DataFrame référencée par 'Contreparties' dans loaded_settings."""
    global cp_box, cp_count
    # étape 1 créer autant de lignes sur dashboard que de valeurs dans df
    if len(list(cp_box.children)) < len(cp_settings): # si pas assez de lignes
        for i in range(len(list(cp_box.children)) + 1, 
                       len(cp_settings) + 1):
            cp_count += 1
            cp_box.children=tuple(list(cp_box.children) + [fw_cp_line(cp_count)])
    elif len(list(cp_box.children)) > len(cp_settings):  # si trop de lignes
        for i in range(len(cp_settings) + 1,
                       len(list(cp_box.children)) + 1):
            if cp_count > 1:
                cp_count -= 1        
                cp_box.children=tuple(list(cp_box.children)[:-1])

    # étape 2 remplir les paramètres
    for i_line, line in enumerate(list(cp_box.children)):
        for i in range(1, len(list(line.children))):  # len permet de prendre en compte poids si implémenté
            if line.children[i].description in cp_settings.columns:
                line.children[i].value = (cp_settings.iloc[i_line]
                                                     .loc[line.children[i].description])
            else:
                raise ValueError(f"Array labels in 'Contreparties' sheet ({list(cp_settings.columns)}) "
                                 f"have no '{line.children[i].description}' label")
    # return quelque chose ?


def load_pcosts(costs_settings):
    """Charge les nouveaux paramètres des contreparties de costs_settings dans vb_fcost_list.
    costs_settings est la DataFrame référencée par 'Contreparties' dans loaded_settings."""
    global vb_fcost_list, fcost_count
    # étape 1 créer autant de lignes sur dashboard que de valeurs dans df
    if len(list(vb_fcost_list.children)) < len(costs_settings): # si pas assez de lignes
        for i in range(len(list(vb_fcost_list.children)) + 1, 
                       len(costs_settings) + 1):
            fcost_count += 1
            vb_fcost_list.children=tuple(list(vb_fcost_list.children) + [fw_fcost_line(fcost_count)])
    elif len(list(vb_fcost_list.children)) > len(costs_settings):  # si trop de lignes
        for i in range(len(costs_settings) + 1,
                       len(list(vb_fcost_list.children)) + 1):
            if fcost_count > 1:
                fcost_count -= 1        
                vb_fcost_list.children=tuple(list(vb_fcost_list.children)[:-1])

    # étape 2 remplir les paramètres
    for i_line, line in enumerate(list(vb_fcost_list.children)):
        for i in range(len(list(list(vb_fcost_list.children)[0].children))):  # = longueur ligne
            if line.children[i].description in costs_settings.columns:
                line.children[i].value = (costs_settings.iloc[i_line]
                                                     .loc[line.children[i].description])
            else:
                raise ValueError(f"Array labels in 'Coûts' sheet ({list(costs_settings.columns)}) "
                                 f"have no '{line.children[i].description}' label")
    # return quelque chose ?


def load_parameters(loaded_settings):
    """Charge les nouveaux paramètres de simulation.
    loaded_settings est un dict {sheet: df}"""
    # chargement paramètre crowd (Palier, Essais, % plateforme et Frais de ports) 
    load_pcrowd(loaded_settings[file_sheets[0]])  # car file_sheets[0] == 'Simulation'
    # chargement paramètre cp (Prix, Coûts, Limite) 
    load_pcp(loaded_settings[file_sheets[1]])  # car file_sheets[1] == 'Contreparties'
    # chargement paramètre costs (Référence, Coûts, + % palier) 
    load_pcosts(loaded_settings[file_sheets[2]])  # car file_sheets[1] == 'Contreparties'

widget de téléversement

In [ ]:
#@title
# widget de téléversement
uploader = wg.FileUpload(
    accept='.xlsx',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    layout=l_20,
    description='Téléversement',
)

# fonction d'observation du bouton de téléversement
def on_upload(change):
    """Fonction d'observation du bouton de téléversement uploader.
    Charge les paramètres de simulation si un fichier est téléversé."""
    global actualise_btn, calc_btn
    reset_load_bar()  # reset load_bar
    reset_calc_btn()  # reset bouton de calcul
    if calc_btn:
        calc_btn.disabled = True  # désactive le bouton de calcul pendant le chargement
    if actualise_btn:
        actualise_btn.disabled = True  # désactive le bouton d'acutalisation
    
    # chargement des données
    loaded_settings = extract_upfile(change['new'])  # on accède directement à uploader.value
    load_parameters(loaded_settings)  # on charge les paramètres

    if calc_btn:
        calc_btn.disabled = False  # réactive le bouton de calcul après le chargement

    # met à jour le profit voulu dans la section d'estimation
    actualise_desired_profit()

uploader.observe(on_upload, names='value')

### Boîte de sauvegarde/chargement des paramètres `box_io`

In [ ]:
#@title
# Label
w_label_io = wg.HTML(
    value=f"<b>Gestion des paramètres :</b>",
    layout=l_20,
    style=l_style,
    )

# séparateur
w_sep2 = wg.HTML(
    value='',
    layout=l_20,
    style=l_style,
    )

# on peut ajouter une barre de chargement à la place de w_sep2
# ou espace pour un message ? -> changer la valeur de w_sep2 ? Output ?
box_io = wg.HBox([w_label_io, uploader, w_sep2, settings_filename, param_download_btn],
                  layout=wg.Layout(justify_content='space-between', border='1px solid'),
                  )

***
## Estimation de palier :

### Classe :

Classe `Predict_container`, qui hérite de `Containers_gen` pour permettre la génération des différents tableaux utilisé pour la prédiction de palier et les opérations associées.

In [ ]:
#@title
class Predict_container(Containers_gen):
    """containers and operations milestone prediction"""

    def success_rate(self, profit):
        """Calcul le taux de success de la simulation sans calculer la full_df"""
        self.df_result()  # calcul full df (élimine les doublons)
        ar_profits = (self.full_df.loc[:, 'profits'])  # extraction des profits
        return (len(ar_profits.loc[ar_profits >= profit])
                /len(ar_profits)) * 100


    def sell_to_profit(self, profit, success_rate=90):
        """Applies the sell_rnd_one method as long as all the trials have not 
        reached the desire profit.
        If the trials_df as already been filled, everything is reset for another
        trial."""
        # reset if needed
        if np.any(self.revenues > 0):
            self.reset_containers()
        # proscess
        process = True
        # calcul de l'incrément du à partir des profits des CPs
        mean_cp_profits = [cp.profit for cp in self.dict_cp.values()]
        increment = round(sum(mean_cp_profits)/len(mean_cp_profits), -1)
        # si incrément trop petit -> 2.5% du profit voulu (limite temps de calcul)
        if increment < 0.025 * profit:
            increment = 0.025 * profit
        # si incrément trop grand -> 10% du profit voulu (les "sauts" trop importants)
        if increment > 0.1 * profit:
            increment = 0.1 * profit
        # palier de départ simulation (ne peut pas être inférieur au profit désiré)
        self.milestone = profit - increment
        # variable de stockage des résultats
        old_ms = new_ms = profit
        old_success = new_success = 0
        # correction (corrige la valeur de succès pour les petits essais)
        correction = 5

        # première itération
        # ==================
        while process:
            # stockage résultat précédent
            old_ms = self.milestone
            old_success = new_success
            # incrémentation
            self.milestone += increment
            # stockage palier actuel
            new_ms = self.milestone
            # vente jusqu'au palier
            self.sell_to_milestone()
            # taux de succès
            new_success = self.success_rate(profit)
            # si supérieur à ce qu'on veut, on arrête
            if (new_success-correction) >= success_rate:
                process = False


        # calcul de la moyenne entre les deux résultats
        mean_ms = (old_ms + new_ms)/2
        self.milestone = mean_ms
        # simulation avec cette valeur
        self.sell_to_milestone()
        # calcul du succès
        mean_success = self.success_rate(profit)

        # calcul du supérieur à la borne haute
        sup_ms = new_ms + (new_ms - old_ms)/2
        self.milestone = sup_ms
        # simulation avec cette valeur
        self.sell_to_milestone()
        # calcul du succès
        sup_success = self.success_rate(profit)

        return [(int(old_ms), old_success), 
                (int(mean_ms), mean_success), 
                (int(new_ms), new_success),
                (int(sup_ms), sup_success),]

### Fonctions

In [ ]:
#@title
# fonctions
# =========

def aug_trial_nb(nb):
    """Double nb si commence par 5, le multiplie par 5 si commence par 1. 
    (retourne nb si ni l'un ni l'autre)"""
    res = nb
    if str(nb)[0] == '1':
        res *= 5
    else:
        res *= 2
    return res


def best_trial_nb(liste_cp, milestone, plateform_part, shipping_fees, min_duplicates=0.5):
    """Trouver un nombre d'essais plus corrects en fonction des paramères donnés."""
    finer_trials_nb = 5_000  # car de toutes manières le nombre minimum d'essais est fixé à 10_000
    trial_dup = 0
    while trial_dup < min_duplicates:  # on réalise des simulation jusqu'à un nombre de doublons suffisants
        # augmentation du nombre d'essais selon la fonction aug_trial_nb()
        finer_trials_nb = aug_trial_nb(finer_trials_nb)
        # génération des contenants
        test_finer = Predict_container(finer_trials_nb, liste_cp, milestone, plateform_part, shipping_fees)
        # vente jusqu'au palier
        test_finer.sell_to_milestone()
        # calcul des résultats
        test_finer.df_result()
        # pour obtenir le pourcentage de doublons
        trial_dup = test_finer.duplicate_percent
    return finer_trials_nb


def final_pslice(tup_list, rate):
    """Ressort un slice de tup_list le plus proche de rate"""
    list_sup = []
    for el in tup_list:
        if el[1] >= rate:
            list_sup.append(el)
    if len(list_sup) <= 2:
        res = tup_list[(3-len(list_sup)):]
    else:
        res = tup_list[:(3-(2*len(list_sup))%6)]
    return res


def pred_text(fpred_list, srate, profit, nb_trials, accuracy=0):
    """Fonction utilisée pour la création du texte des résultats d'estimation"""
    dict_acc = {0: '', 0.25: 'rapide', 0.5: 'équilibrée', 0.75: 'précise', 0.90:'extrème'}
    res = (f" Estimation {dict_acc[accuracy]} du palier optimal avec {srate} % de succès"
           f"* pour {profit} € de profits minimum*\n sur {nb_trials:,} essais :\n"
           f"\n Ce palier devrait être ")
    if len(fpred_list) >= 2:
        res += (f"compris entre {fpred_list[0][0]} € ({fpred_list[0][1]:.2f} %) "
                f"et {fpred_list[-1][0]} € ({fpred_list[-1][1]:.2f} %)@")
        if len(fpred_list) == 3:
            res += f"\n Essayez {fpred_list[1][0]} € ({fpred_list[1][1]:.2f} %)@"
    else:
        if fpred_list[0][1] >= srate:
            res += f"aux alentours de "
        else:
            res += f"supérieur à "
        res += f"{fpred_list[0][0]} € ({fpred_list[0][1]:.2f} %)@"
    # les chiffres sont représenté au format anglo-saxon (1,000.02)
    # traduction en fr :
    res = res.replace(',', ' ')
    res = res.replace('.', ',')
    res = res.replace('@', '.')
    res = res.replace('*', ',')
    return res


def pred_watchdog_timer(ini_t, timer=10):
    """Fonction permettant de contrôler le temps d'exécution en lui passant
    le temps initial et une limite (timer, en minutes)"""
    res = None
    limit = timer * 60
    current = time.time()
    if (current - ini_t) > limit:
        res = (f'Watchdog timer - limite de {timer} minutes dépassée, choisissez'
               f' une précision moins élevée.')
    return res

### Widgets :

In [ ]:
#@title
# label de la boîte des coûts
pred_label = wg.HTML(
    value=f"<b>Estimation de palier :</b>",
    layout=l_20,
    style=l_style,
    )

# sélection du profit voulu
w_desired_profit = wg.BoundedIntText(
        value=1_200, min=0, 
        max=1_000_000_000,
        step=50,
        description="Profit voulu",
        layout=l_20,
        style=l_style,
        disabled=False,
        description_tooltip=('Profit voulu pour l\'estimation du palier. '
                             'Valeur automatiquement mise à jour lors d\'un clic sur les boutons '
                             '\'Calculer\' ou \'Actualiser\', ou lors du chargement d\'un fichier de paramètres'),
        )

# sélection du succès voulu
w_desired_srate = wg.BoundedFloatText(
        value=90, min=0, max=100, step=0.5,
        description='Taux de succès',
        layout=l_20,
        style=l_style,
        disabled=False,
        description_tooltip=('Taux de succès voulu pour lequel le palier doit être estimé.'),
        )


# sélection de la précision
w_accuracy = wg.Dropdown(
    options=[('rapide', 0.25), ('équilibrée', 0.5), ('précise', 0.75), ('extrème (non-recommandé)', 0.90)],
    value=0.5,
    description='Analyse',
    layout=l_20,
    style=l_style,
    disabled=False,
    description_tooltip=('Précision voulue pour l\'estimation du palier. '
                         'Plus la précision augmente, plus la vitesse de calcul diminue. '
                         'La précision \'extrème\' peut prendre plusieurs minutes et est '
                         'déconseillée si l\'analyse \'précise\' dure déjà plus d\'une minute.'
                         ),
    )

# sortie des infos
out_pred = wg.Output(layout=wg.Layout(width='100%',
                                      justify_content='center',
                                      ))

fonction d'actualisation de `w_desired_profit`

In [ ]:
#@title
def actualise_desired_profit(widgt=w_desired_profit):
    """Calcule automatiquement la valeur du profit voulu (w_desired_profit) en fonction du palier et des coûts rentrés.
    Est appelée dans :
    - on_calc_clicked()
    - on_actualise_clicked()
    - on_upload()"""
    fcost_val = []  
    pcost_val = []
    for line in list(vb_fcost_list.children):
        fcost_val.append(line.children[1].value)
        pcost_val.append(line.children[2].value)
    widgt.value = sum(fcost_val) + (sum(pcost_val)/100) * w_milestone.value

### Bouton pour le calcul de l'estimation et fonctions associées :

In [ ]:
#@title
# bouton de calcul
predict_btn = wg.Button(
    description = 'Estimation', 
    button_style='',
    tooltip="Estimation du palier optimal.",
    layout=l_20,
    )

# fonction de reset du bouton
def reset_predict_btn(b):
    """Remet à l'état initial le bouton d'estimation."""
    b.description = 'Estimation'
    b.button_style=''
    b.tooltip="Estimation du palier optimal."
    b.disabled = False

# fonction travail du bouton
def working_predict_btn(b):
    """Etat de travail du bouton d'estimation."""
    b.description = 'En cours'
    b.button_style=''
    b.tooltip="Estimation en cours."
    b.disabled = True

# déclaration container
contain = None

# fonction associée
def on_predict_clicked(b):
    global w_desired_profit, w_desired_srate, w_accuracy

    # Nettoyage sortie
    out_pred.clear_output()
    with out_pred:
        print(' Initialisation ...', end='')
    
    # watchdog timer (en minutes)
    watch_time = 10

    # état de travail du bouton
    working_predict_btn(b)
    
    # récupération valeurs
    # ====================
    desired_profit = w_desired_profit.value
    desired_srate = w_desired_srate.value
    min_duplicates = w_accuracy.value

    # constante : nombre d'essais utilisés pour la première estimation
    pred_trials_nb = 5_000
    # compteur d'étape (pour barre de chargement)
    step = 0
    init_time = step_time = time.time()

    # paramètres crowd
    plateform_part = w_plateform_part.value / 100
    shipping_fees = w_shipping_fees.value

    # contreparties :
    liste_cp = []  # liste des CP de générés avec la classe couterparts
    for line in list(cp_box.children):
        liste_cp.append(Counterpart(line.children[1].value, 
                                    line.children[2].value, 
                                    line.children[3].value,))

    # CORE
    # ====
    
    # etape 1: prédiction grossière (la valeur du palier n'est pas importante -> fixée à 2_000)
    gross_pred = Predict_container(pred_trials_nb, liste_cp, 2_000, plateform_part, shipping_fees)
    # liste des tuples (paliers possibles, taux de réussite)
    list_gp = gross_pred.sell_to_profit(desired_profit, desired_srate)
    # étape(temporaire)
    step += 1
    old_step_time = step_time
    step_time = time.time()
    out_pred.clear_output()
    with out_pred:
        print(f' Etape {step} - temps d\'exécution : {step_time-old_step_time:.0f} s')
    # watchdog timer
    exced_watchdog = pred_watchdog_timer(init_time, watch_time)
    if exced_watchdog:
        reset_predict_btn(b)
        with out_pred:
            print(exced_watchdog)
        return

    # etape 2: trouver un nombre d'essais plus corrects, utilise le dernier palier trouvé
    finer_trials_nb = best_trial_nb(liste_cp, list_gp[-1][0], 
                                    plateform_part, 
                                    shipping_fees, 
                                    min_duplicates)
    # étape(temporaire)
    step += 1
    old_step_time = step_time
    step_time = time.time()
    #out_pred.clear_output()
    with out_pred:
        print(f' Etape {step} - temps d\'exécution : {step_time-old_step_time:.0f} s')
    # watchdog timer
    exced_watchdog = pred_watchdog_timer(init_time, watch_time)
    if exced_watchdog:
        reset_predict_btn(b)
        with out_pred:
            print(exced_watchdog)
        return

    # etape 3 à 6: affinage des résultats
    finer_res = []
    for predict in list_gp:  # on recalcule toutes les prédictions avec le nombre d'essais trouvés à l'étape 2
        finer_pred = Predict_container(finer_trials_nb, liste_cp, predict[0], plateform_part, shipping_fees)
        finer_pred.sell_to_milestone()
        finer_res.append((predict[0], finer_pred.success_rate(desired_profit)))
        # étape(temporaire)
        step += 1
        old_step_time = step_time
        step_time = time.time()
        #out_pred.clear_output()
        with out_pred:
            print(f' Etape {step} - temps d\'exécution : {step_time-old_step_time:.0f} s')
        # watchdog timer
        exced_watchdog = pred_watchdog_timer(init_time, watch_time)
        if exced_watchdog:
            reset_predict_btn(b)
            with out_pred:
                print(exced_watchdog)
            return

    # etape 7: slice sur les résultats affinés
    fpslice = final_pslice(finer_res, desired_srate)
    # étape(temporaire)
    step += 1
    old_step_time = step_time
    step_time = time.time()
    #out_pred.clear_output()
    with out_pred:
        print(f' Etape {step} - temps d\'exécution : {step_time-old_step_time:.0f} s')

    # etape 8: transformation en text
    out_pred.clear_output()
    with out_pred:
        print(pred_text(fpslice, desired_srate, desired_profit, finer_trials_nb, min_duplicates))
    # ici on fait un print, il faudra le capturer
    # étape(temporaire)
    step += 1
    old_step_time = step_time
    step_time = time.time()
    with out_pred:
        print(f'\n     (temps d\'exécution : {step_time-init_time:.0f} s)')

    # bouton à l'état normal
    reset_predict_btn(b)

predict_btn.on_click(on_predict_clicked)

### Boîte GUI `pred_box` :

In [ ]:
#@title
pred_line1 = wg.HBox(
    [pred_label, w_desired_profit, w_desired_srate, w_accuracy, predict_btn],
    layout=wg.Layout(justify_content='space-between', border='1px solid'))

pred_box = wg.VBox(
    [pred_line1,
     out_pred],
    layout=wg.Layout(border='1px solid'))

***
## Graphs de correlations

### Génération du graph

#### Fonction de corrélation

In [ ]:
#@title
def corelation(lv_mean, tolerance=0.05):
    """Décrit la corelation entre deux columns d'une dataframe à partir de 
    l'évolution des valeurs moyennes lv_mean avec une tolérance spécifiée"""
    list_val = [lv_mean[round(x * (len(lv_mean)-1) / 4)] for x in range(0, 5)]
    val_max = np.max(lv_mean)  # pour la tolérance autour de 0
    # calcul d'une valeur de tolérance
    count_corr = 0
    for i in range(4):
        if list_val[i + 1] - list_val[i] > val_max * tolerance:
            count_corr += 1
        elif list_val[i + 1] - list_val[i] < - val_max * tolerance:
            count_corr -= 1
    
    # tranformation en text
    res = 'corrélation'
    if count_corr > 0:
        res += ' positive'
    elif count_corr < 0:
        res += ' négative'
    else:
        res = 'peu ou pas de ' + res

    if abs(count_corr) >= 3:
        res = 'forte ' + res

    return res

#### Fonction de génération et d'affichage des labels

In [ ]:
#@title
def label_from_column(column, on_axis=False):
    """Crée les labels des axes à partir des noms des columns d'une dataframe"""
    res = ''
    if 'CP_' in column:
        res += 'ventes de la contrepartie ' + column[3:]
    elif column == 'sells':
        res += 'ventes'
    elif column == 'costs':
        res += 'coûts'
        if on_axis:
            res += ' (€)'
    elif column == 'profits':
        res += 'profits'
        if on_axis:
            res += ' (€)'
    if on_axis:
        res = res.capitalize()
    return res

#### Fonction d'affichage du graph

In [ ]:
#@title
def plot_hl_envelopes(column_x, column_y):
    """
    trace les enveloppes des courbes de correlations entre les column_x et column_y
    de la dataframe df
    """
    global contain
    # valeurs de lissage des enveloppes
    dmin=15
    dmax=15
    dmean=7

    plt.close('all') 

    if contain and column_x and column_y:
        # préparation des données
        # =======================
        df = contain.full_df
        ar_max = np.array(df.groupby(column_x).max().loc[:, column_y])
        ar_min = np.array(df.groupby(column_x).min().loc[:, column_y])
        ar_mean = np.array(df.groupby(column_x).mean().loc[:, column_y])

        # locals min      
        lmin = (np.diff(np.sign(np.diff(ar_min))) > 0).nonzero()[0] + 1 
        # locals max
        lmax = (np.diff(np.sign(np.diff(ar_max))) < 0).nonzero()[0] + 1 
        

        # global max of dmax-chunks of locals max 
        lmin = lmin[[i+np.argmin(ar_min[lmin[i:i+dmin]]) for i in range(0,len(lmin),dmin)]]
        # global min of dmin-chunks of locals min 
        lmax = lmax[[i+np.argmax(ar_max[lmax[i:i+dmax]]) for i in range(0,len(lmax),dmax)]]
        # lissage moyenne
        def smooth(y, box_pts):
            """lisse un signal, concerve le même nombre de points"""
            box = np.ones(box_pts)/box_pts
            y_smooth = np.convolve(y, box, mode='same')
            return y_smooth
        vmean = smooth(ar_mean, dmean)  # c'est un tableau
        # index des valeurs moyennes
        #idx_mean = sorted(set(list(lmin) + list(lmax)))

        # ajout des extrémums
        def add_extrem(arr):
            arr = np.insert(arr, 0, 0)
            arr = np.append(arr, -1)
            return arr
        lmin = add_extrem(lmin[1: -1])
        lmax = add_extrem(lmax[1: -1])
        # valeurs moyennes
        vmean[0] = ar_mean[0]
        vmean[-1] = ar_mean[-1]
        #idx_mean = add_extrem(idx_mean[1: -1])

        # affichage
        # =========
        fig, pline1 = plt.subplots(1, 3, 
                                   figsize=(19.6, 4.8),
                                   constrained_layout=True)
        
        # première ligne de plots :
        (l_ax1, m_ax1, r_ax1) = pline1

        x = df.groupby(column_x).max().index

        #df.plot(x=column_x, y=column_y, ax=ax) # pour tracer le signal
        m_ax1.plot(x[lmax], ar_max[lmax], 'g', label='hautes')
        m_ax1.plot(x[lmax], vmean[lmax], 'y', label='moyennes')
        m_ax1.plot(x[lmin], ar_min[lmin], 'r', label='basses')
        m_ax1.set_title(f"Profil des {label_from_column(column_y)} en fonction des {label_from_column(column_x)}\n"
                    f" générés ({corelation(vmean[lmax])})")
        m_ax1.set_xlabel(f"{label_from_column(column_x, on_axis=True)}")
        m_ax1.set_ylabel(f"{label_from_column(column_y, on_axis=True)}")
        m_ax1.legend(title='Valeurs')

        # autres axes
        l_ax1.set_axis_off()
        r_ax1.set_axis_off()

        fig.savefig('graph_corr.png')
        save_corr_btn.disabled = False  # active le bouton de sauvegarde
        save_corr_btn.tooltip="Sauvegarde le graphique de corrélation au format png."

        plt.show();

### Tableau de bord

#### Widgets basiques

In [ ]:
#@title
# Label
w_label_corr = wg.HTML(
    value=f"<b>Graphs de corrélation :</b>",
    layout=l_25,
    style=l_style,
    )

# sélection des x ('profits' ou 'costs')
w_corr_xaxis = wg.Dropdown(
    options=[('Profits', 'profits'), ('Coûts', 'costs')],
    value=None,
    description='Abscisse',
    layout=l_25,
    style=l_style,
    disabled=True,  # activé si données chargée (cf on_actualise_change_corrd())
    description_tooltip=('Paramètre représenté en abscisse.'
                         ),
    )

# sélection des y ('CP_X' ou 'sells')
w_corr_yaxis = wg.Dropdown(
    options=[('Ventes', 'sells')],
    value=None,
    description='Ordonnée',
    layout=l_25,
    style=l_style,
    disabled=True,  # activé si données chargée (cf on_actualise_change_corrd())
    description_tooltip=('Paramètre représenté en ordonnée.'
                         ),
    )

# désactivation si actualise_btn est désactivé
def on_actualise_change_corrd(change):
    global contain
    if change['new']:
        # dropdown des x
        w_corr_xaxis.disabled = change['new']
        w_corr_xaxis.value=None
        # dropdown des y
        w_corr_yaxis.disabled = change['new']
        w_corr_yaxis.options=[('Ventes', 'sells')]
        w_corr_yaxis.value=None
        # bouton de sauvegarde 
        save_corr_btn.disabled = True
        save_corr_btn.tooltip = tooltip="Générez d'abord le graphique."
    else:
        # dropdown des x
        w_corr_xaxis.disabled = change['new']
        # dropdown des y
        w_corr_yaxis.disabled = change['new']
        corr_opt_list = [(f"Contrpartie {x+1}", f"CP_{x+1}") for x in range(len(contain.dict_cp))]
        corr_opt_list.append(('Ventes', 'sells'))
        w_corr_yaxis.options=corr_opt_list  # ajouter toutes les contreparties
        w_corr_yaxis.value=None
actualise_btn.observe(on_actualise_change_corrd, names='disabled')

Bouton de téléchargement d'image

In [ ]:
#@title
# interaction entre les widgets et la fonction d'affichage
save_corr_btn =  wg.Button(
    description = 'Save png', 
    button_style='',
    tooltip="Générez d'abord le graphique.",
    layout=l_25,
    disabled=True, # activé si graph généré
)
# cf graph_corr() pour activation et 


def on_savecorr_clicked(b):
    global contain, out_graph

    # tracé
    if contain:
        files.download('graph_corr.png')
        b.button_style='success'
        time.sleep(1)
        b.button_style=''
    else:
        b.button_style='danger'
        with out_graph:
            print(' Erreur : les données n\'ont pas été générées.')
        time.sleep(1)
        b.button_style=''

save_corr_btn.on_click(on_savecorr_clicked)

#### interactive_output

In [ ]:
#@title
out_corr = wg.interactive_output(plot_hl_envelopes, {'column_x': w_corr_xaxis, 'column_y': w_corr_yaxis})    

### Boîte des graph de corrélations `box_corr`

In [ ]:
#@title
# boîte de sélection des données
box_select_corr_ = wg.HBox([w_label_corr, w_corr_xaxis, w_corr_yaxis, save_corr_btn
                           ], layout=wg.Layout(justify_content='space-between', border='1px solid'))

# box_graph
box_corr = wg.VBox([box_select_corr_,
                     wg.HBox([out_corr], layout=l_center),
                     ],
                   layout=wg.Layout(border='1px solid'))

***
##  Tableau de bord complet (`dashboard`) :

In [ ]:
#@title
dashboard = wg.VBox([box_df,
                     box_graph,
                     box_corr,  # voir enchainement ?
                     pred_box,
                     box_io,
                     ],
                    layout=wg.Layout(border='1px solid'),
                    )

***
# **Affichage** :

In [ ]:
#@title
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 10000})'''))
dashboard

<IPython.core.display.Javascript object>